In [1]:
from urllib.request import urlopen
from urllib.parse import quote_plus as qp #문자 아스키코드로 변환
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time
import os

#인터넷 창 띄우기
driver = webdriver.Chrome()

WebDriverException: Message: unknown error: Failed to create Chrome process.
Stacktrace:
Backtrace:
	Ordinal0 [0x0116B8F3+2406643]
	Ordinal0 [0x010FAF31+1945393]
	Ordinal0 [0x00FEC748+837448]
	Ordinal0 [0x01008702+952066]
	Ordinal0 [0x01005D71+941425]
	Ordinal0 [0x01038EE0+1150688]
	Ordinal0 [0x01038B3A+1149754]
	Ordinal0 [0x01034096+1130646]
	Ordinal0 [0x0100E636+976438]
	Ordinal0 [0x0100F546+980294]
	GetHandleVerifier [0x013D9612+2498066]
	GetHandleVerifier [0x013CC920+2445600]
	GetHandleVerifier [0x01204F2A+579370]
	GetHandleVerifier [0x01203D36+574774]
	Ordinal0 [0x01101C0B+1973259]
	Ordinal0 [0x01106688+1992328]
	Ordinal0 [0x01106775+1992565]
	Ordinal0 [0x0110F8D1+2029777]
	BaseThreadInitThunk [0x75F6FA29+25]
	RtlGetAppContainerNamedObjectPath [0x775D7A7E+286]
	RtlGetAppContainerNamedObjectPath [0x775D7A4E+238]


In [3]:
search = '평남식당'

baseurl = 'https://search.daum.net/search?w=img&nil_search=btn&DA=NTB&enc=utf8&q='
url = baseurl + qp(search)
driver.get(url)

imgurl=[] #이미지 url 저장 list

In [3]:
scroll_num = 100 #스크롤 횟수
scroll_pause_time=1.5 #스크롤 내릴때 이미지 로딩되는 시간 기다림
upscroll=-100
pic_num = 100 # 최대로 가져올 사진의 수
#스크롤 하기
#스크롤은 기다리기 귀찮으면 직접 한 후에 아래 코드 실행해도 됨
#인터넷 환경에 따라 이미지 로딩이 느릴때, 스크롤이 적게 실행될 수 있음
breaker = [] # 업데이트 5회이상 안될시, break


for i in range(scroll_num+1):

    #파싱
    html = driver.page_source
    soup = BeautifulSoup(html)
    daum = soup.find_all(class_='thumb_img')
    for pic in daum:
        imgurl.append(pic['src']) #이미지 주소 가져오기
        imgurl = (list(set(imgurl))) #이미지주소 중복 제거
    
    print('스크롤 {}번 실행, {}개 그림을 가져올 수 있습니다.'.format(i, len(list(set(imgurl)))))
    breaker.append(len(list(set(imgurl))))
    time.sleep(scroll_pause_time)
    last_height = driver.execute_script("return document.body.scrollHeight")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)
    new_height = driver.execute_script("return document.body.scrollHeight")
    driver.execute_script('scrollBy(0,{})'.format(upscroll))
    
    if new_height == last_height:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause_time)
        new_height = driver.execute_script("return document.body.scrollHeight")
        driver.execute_script('scrollBy(0,{})'.format(upscroll))
        if new_height == last_height:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(scroll_pause_time)
            new_height = driver.execute_script("return document.body.scrollHeight")
            driver.execute_script('scrollBy(0,{})'.format(upscroll))
        else:
            last_height = new_height
            continue
    if len(breaker)>5 and breaker[-3]==breaker[-1]: 
        print('더이상 update가 없습니다.')
        break
    if breaker[-1] >= pic_num:
        print(f'사진의 수가 {breaker[-1]}개로 최대개수인 {pic_num}개 이상입니다')
        break

스크롤 0번 실행, 80개 그림을 가져올 수 있습니다.
스크롤 1번 실행, 160개 그림을 가져올 수 있습니다.
사진의 수가 160개로 최대개수인 100개 이상입니다


In [4]:
num = len(os.listdir('original_pictures')) #class 갯수
os.mkdir('./original_pictures/'+'{}_'.format(num+1)+search)  #폴더 생성
for i, pic in enumerate(imgurl):
    with urlopen(pic) as f: #날짜 시간 포함하여 이미지 저장
        with open('./original_pictures/'+'{}_'.format(num+1)+search+'/'+search+'_'+'{0:04d}'.format(i+1)+'_'+time.strftime('%y%m%d-%H%M')+'.jpg','wb') as h:
            img = f.read()
            h.write(img)
    print('이미지 저장 중 : {}/{}'.format(i+1,len(imgurl)), end='\r', flush=True)